In [1]:
from time import sleep
import google.generativeai as genai
import configparser
import json
import re
import pandas as pd

/home/gcasado0/proyectos/chatbot/chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# leer archivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')
GOOGLE_API_KEY = config['GOOGLE']['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [4]:
consulta = """
Que no soy oriundo de la ciudad de Rosario, de modo circunstancial me encontraba circulando por la ciudad por lo que puede
 que haya cometido alguna infracción según las ordenanzas locales, por desconocimiento. Solicito se aplique una reducción en la pena aplicada a los fines de poder hacer frente a la misma.
"""

consulta = """
Se realizan dos multas del mismo día, en el mismo lugar con diferencia horaria, utilizando en una la foto delantera del auto y en la segunda la foto trasera. Se está cobrando la misma infracción dos veces utilizando dos fotos diferentes donde se ve que el auto está en el mismo lugar en ambas.
"""

prompt = f"""
Se le proporcionarán descargos de vecinos por multas al sitio web de la municipalidad.
En base al siguiente texto, que está delimitado por tres comillas invertidas, determine si las personas quieren pagar la multa o bien, se están justificando o excusandose de la infracción.
Indique el sentimiento general del texto.
Texto: '''{consulta}'''

El formato de salida es JSON, con los siguientes campos:
"Quiere pagar": "Si/No"
"Quiere justificar": "Si/No"
"Sentimientos": 
"""
response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
    )

print(response.text)

```json
{
  "Quiere pagar": "No",
  "Quiere justificar": "Si",
  "Sentimiento": "Indignado/Reclamatorio"
}
```



Recorrer la lista de consultas de habilitaciones y obtener tematicas claves y sentimientos de las mismas


In [5]:
data = pd.read_excel('Descargos Ene-Feb 2025.xlsx')
data.head(5)

,tramite_tipo_id,codigo_gestion,solicitante_cuit,destinatario_cuit,fecha_presentacion,fecha_fin,descargo,patente
0,129,79512-2025,20140824756,20140824756,2025-02-06 08:23:37.196,2025-02-11 13:13:42.14,"""Que no soy oriundo de la ciudad de Rosario, d...","""AF365QY"""
1,75,77568-2025,27345723981,27345723981,2025-02-05 15:23:39.688,2025-02-07 12:13:09.222,"""Se realizan dos multas del mismo día, en el m...","""AF682GM"""
2,75,79556-2025,20206773260,20206773260,2025-02-06 09:43:37.703,2025-02-12 11:23:44.889,"""El día 29/01/2025, de confección del acta, el...","""AG956JC"""
3,75,69997-2025,20438423118,20438423118,2025-02-03 10:39:51.8,2025-02-07 08:24:55.675,"""Buenos días, me acaban de echar del trabajo y...","""A111ECW"""
4,75,65844-2025,20174607096,20174607096,2025-01-31 20:23:44.277,2025-02-05 12:40:42.898,"""Solicito la anulación de la infracción.\r\nEs...","""AC906BQ """


In [9]:
# recorrer todo el dataframe data y extraer la intencion y sentimientos

for i, row in data.iterrows():

    consulta = data['descargo'][i]
    # extraer comillas dobles de la consulta
    consulta = re.sub(r'\"', '', consulta)

    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 
    print(consulta)
    prompt = f"""
    Se le proporcionarán descargos de vecinos por multas al sitio web de la municipalidad.
    En base al siguiente texto, que está delimitado por tres comillas invertidas, determine si las personas quieren pagar la multa o bien, se están justificando o excusandose de la infracción.
    Indique el sentimiento general del texto.
    Texto: '''{consulta}'''

    El formato de salida es JSON, con los siguientes campos:
    "Quiere pagar": "Si/No"
    "Quiere justificar": "Si/No"
    "Sentimientos": 
    """
    
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Quiere pagar'] = analisis.get('Quiere pagar', 'None')
            data.loc[i, 'Quiere justificar'] = analisis.get('Quiere justificar', 'None')            
            data.loc[i, 'Sentimientos'] = ', '.join(analisis.get('Sentimientos', [])) if isinstance(analisis.get('Sentimientos', []), list) else analisis.get('Sentimientos', 'None')

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")

    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
    if i>300:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
data.to_excel('Descargos-procesado.xlsx', index=False)


Que no soy oriundo de la ciudad de Rosario, de modo circunstancial me encontraba circulando por la ciudad por lo que puede que haya cometido alguna infracción según las ordenanzas locales, por desconocimiento. Solicito se aplique una reducción en la pena aplicada a los fines de poder hacer frente a la misma.
```json
{
  "Quiere pagar": "Si",
  "Quiere justificar": "Si",
  "Sentimientos": ["Solicitante", "Sumiso", "Con arrepentimiento", "Desesperado por una solución"]
}
```

Se realizan dos multas del mismo día, en el mismo lugar con diferencia horaria, utilizando en una la foto delantera del auto y en la segunda la foto trasera. Se está cobrando la misma infracción dos veces utilizando dos fotos diferentes donde se ve que el auto está en el mismo lugar en ambas.
```json
{
  "Quiere pagar": "No",
  "Quiere justificar": "Si",
  "Sentimientos": ["Indignación", "Furia", "Inconformidad"]
}
```

El día 29/01/2025, de confección del acta, el vehículo se encontraba en calle Paraguay al 1100, r

In [ ]:
data = pd.read_excel('encontraste_lo_que_buscabas-procesado.xlsx')
data.head(5)

In [ ]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data.iterrows():

    consulta = data['Mensaje'][i]
    url_origen = data['URL'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 


    # Expresión regular para capturar desde "/inicio/" hasta antes de los parámetros
    pattern = r"/inicio/[^?]+"

    # Buscar el patrón en la URL
    match = re.search(pattern, url_origen)

    # Obtener el resultado
    if match:
        url_result = match.group(0)
    else:
        url_result = url_origen

    print(i, url_result)
    print(consulta)
    prompt = f"""
    Se le proporcionarán consultas de vecinos al sitio web de la municipalidad.
    Clasifique cada consulta en una categoría principal y una categoría secundaria en base al texto y a la URL de origen, que está delimitado por tres comillas invertidas.
    Categorías principales: Reclamos, Multas, Licencia de Conducir, Turnos, Impuestos, Perfil Digital, Salud, Indefinido

    Categorías secundarias de Reclamos:
    Alumbrado
    Arbolado
    Higiene urbana
    Indefinido 

    Categorías secundarias de Multas:
    Pagar
    Turnos
    Titularidad
    Descargos
    Indefinido

    Categorías secundarias de Licencia de conducir:
    Turnos
    Consultas
    Cenat y/o Sellados
    Indefinido

    Categorías secundarias de Impuestos:
    TGI
    Patente
    Indefinido

    Categorías secundarias de Salud:
    Sin turnos
    Error en el sistema
    Reclamo
    Indefinido

    Categorías secundarias de Perfil Digital:
    Actualización de datos
    Indefinido
    
    Categorías secundarias de Salud:
    Vacunas
    Indefinido
    
    Texto: '''{consulta}'''
    URL origen: '''{url_result}'''
    
    Salida en formato JSON:
    "Principal":
    "Secundaria": 
    """
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Principal'] = analisis.get('Principal', 'None')
            data.loc[i, 'Secundaria'] = analisis.get('Secundaria', [])            

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")
        
    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
            
    if i>300:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
data.to_excel('encontraste_lo_que_buscabas-procesado.xlsx', index=False)
